In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle 

In [27]:
sales_df_original = pd.read_csv('validation.csv')
ID_vs_sales = pd.read_csv('store_ID_vs_sales.csv')
ID_vs_sales.drop(columns=['Unnamed: 0'], inplace=True)

In [28]:
sales_df_original.rename(columns={'index': 'Index_Data'}, inplace=True)
sales_df_original

,Index_Data,store_ID,day_of_week,date,nb_customers_on_day,open,promotion,state_holiday,school_holiday
0,272371,415,7,01/03/2015,0,0,0,0,0
1,558468,27,7,29/12/2013,0,0,0,0,0
2,76950,404,3,19/03/2014,657,1,1,0,0
3,77556,683,2,29/01/2013,862,1,0,0,0
4,456344,920,3,19/03/2014,591,1,1,0,0
...,...,...,...,...,...,...,...,...,...
71200,59062,441,7,26/10/2014,0,0,0,0,0
71201,687449,377,7,18/08/2013,0,0,0,0,0
71202,207393,15,3,11/06/2014,648,1,0,0,0
71203,233378,950,2,23/04/2013,626,1,1,0,0


In [29]:
sales_df = sales_df_original[sales_df_original ['open']!=0]

In [30]:
sales_df_transformed = pd.get_dummies(sales_df, columns=["state_holiday", "day_of_week"], prefix=["state_holiday","DoW"], dtype=int,drop_first=True)


In [31]:
sales_df_transformed["date"] = pd.to_datetime(sales_df_transformed["date"], dayfirst=True).apply(lambda x: x.toordinal())

In [32]:
sales_df_transformed_boost = pd.merge(left = sales_df_transformed, right= ID_vs_sales, left_on = 'store_ID', right_on = 'store_ID', how='left' )

In [33]:
sales_df_transformed_boost

,Index_Data,store_ID,date,nb_customers_on_day,open,promotion,school_holiday,state_holiday_a,state_holiday_b,state_holiday_c,DoW_2,DoW_3,DoW_4,DoW_5,DoW_6,DoW_7,sales_per_customer_and_id
0,76950,404,735311,657,1,1,0,0,0,0,0,1,0,0,0,0,8.313361
1,77556,683,734897,862,1,0,0,0,0,0,1,0,0,0,0,0,10.301936
2,456344,920,735311,591,1,1,0,0,0,0,0,1,0,0,0,0,9.869986
3,436466,758,735410,569,1,0,0,0,0,0,0,0,1,0,0,0,7.605707
4,646251,563,735645,321,1,1,0,0,0,0,0,0,0,0,0,0,7.817633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59100,63935,55,735444,583,1,1,1,0,0,0,0,1,0,0,0,0,8.125002
59101,492885,231,734900,378,1,0,0,0,0,0,0,0,0,1,0,0,11.948481
59102,207393,15,735395,648,1,0,0,0,0,0,0,1,0,0,0,0,9.980807
59103,233378,950,734981,626,1,1,0,0,0,0,1,0,0,0,0,0,10.097598


In [34]:
X = sales_df_transformed_boost.drop(columns = ['Index_Data', 'store_ID', 'open']) 
X

,date,nb_customers_on_day,promotion,school_holiday,state_holiday_a,state_holiday_b,state_holiday_c,DoW_2,DoW_3,DoW_4,DoW_5,DoW_6,DoW_7,sales_per_customer_and_id
0,735311,657,1,0,0,0,0,0,1,0,0,0,0,8.313361
1,734897,862,0,0,0,0,0,1,0,0,0,0,0,10.301936
2,735311,591,1,0,0,0,0,0,1,0,0,0,0,9.869986
3,735410,569,0,0,0,0,0,0,0,1,0,0,0,7.605707
4,735645,321,1,0,0,0,0,0,0,0,0,0,0,7.817633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59100,735444,583,1,1,0,0,0,0,1,0,0,0,0,8.125002
59101,734900,378,0,0,0,0,0,0,0,0,1,0,0,11.948481
59102,735395,648,0,0,0,0,0,0,1,0,0,0,0,9.980807
59103,734981,626,1,0,0,0,0,1,0,0,0,0,0,10.097598


In [35]:
from sklearn.preprocessing import StandardScaler

scaler_standard = StandardScaler()
X_standard = scaler_standard.fit_transform(X)



In [36]:
with open('XGBRegressor.pkl', 'rb') as archivo:
    xgb_regressor = pickle.load(archivo)

In [37]:
y_pred = xgb_regressor.predict(X_standard)

In [43]:
prediction = pd.DataFrame(y_pred) 

prediction.rename(columns={0: 'Prediction'}, inplace=True)



In [44]:
prediction

,Prediction
0,5879.517090
1,7904.483398
2,6444.871582
3,4052.584473
4,3116.645264
...,...
59100,5224.025879
59101,4474.070801
59102,6016.455078
59103,6498.030762


In [52]:
df_final = pd.concat([sales_df_transformed_boost, prediction], axis=1)

In [53]:
df_final

,Index_Data,store_ID,date,nb_customers_on_day,open,promotion,school_holiday,state_holiday_a,state_holiday_b,state_holiday_c,DoW_2,DoW_3,DoW_4,DoW_5,DoW_6,DoW_7,sales_per_customer_and_id,Prediction
0,76950,404,735311,657,1,1,0,0,0,0,0,1,0,0,0,0,8.313361,5879.517090
1,77556,683,734897,862,1,0,0,0,0,0,1,0,0,0,0,0,10.301936,7904.483398
2,456344,920,735311,591,1,1,0,0,0,0,0,1,0,0,0,0,9.869986,6444.871582
3,436466,758,735410,569,1,0,0,0,0,0,0,0,1,0,0,0,7.605707,4052.584473
4,646251,563,735645,321,1,1,0,0,0,0,0,0,0,0,0,0,7.817633,3116.645264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59100,63935,55,735444,583,1,1,1,0,0,0,0,1,0,0,0,0,8.125002,5224.025879
59101,492885,231,734900,378,1,0,0,0,0,0,0,0,0,1,0,0,11.948481,4474.070801
59102,207393,15,735395,648,1,0,0,0,0,0,0,1,0,0,0,0,9.980807,6016.455078
59103,233378,950,734981,626,1,1,0,0,0,0,1,0,0,0,0,0,10.097598,6498.030762


In [59]:
temporary_result = df_final[['Index_Data', 'Prediction']]

In [60]:
temporary_result

,Index_Data,Prediction
0,76950,5879.517090
1,77556,7904.483398
2,456344,6444.871582
3,436466,4052.584473
4,646251,3116.645264
...,...,...
59100,63935,5224.025879
59101,492885,4474.070801
59102,207393,6016.455078
59103,233378,6498.030762


In [61]:

final_result = pd.merge(left = sales_df_original, right = temporary_result, left_on = 'Index_Data', right_on = 'Index_Data', how = 'outer')

In [62]:
final_result

,Index_Data,store_ID,day_of_week,date,nb_customers_on_day,open,promotion,state_holiday,school_holiday,Prediction
0,7,764,4,26/12/2013,0,0,0,c,1,NaN
1,19,22,3,22/05/2013,449,1,0,0,1,3713.338135
2,31,1087,6,29/06/2013,622,1,0,0,0,5794.836914
3,45,139,6,17/08/2013,314,1,0,0,0,3434.285889
4,56,568,1,07/04/2014,356,1,0,0,0,3878.811279
...,...,...,...,...,...,...,...,...,...,...
71200,712004,217,2,13/01/2015,633,1,1,0,0,4987.066406
71201,712018,604,3,30/04/2014,743,1,1,0,0,11249.125000
71202,712020,1021,5,18/07/2014,1852,1,1,0,1,15737.120117
71203,712023,28,3,27/08/2014,0,0,0,0,1,NaN


In [63]:
final_result.fillna(0, inplace=True)

In [65]:
output_for_David = final_result[['Index_Data', 'Prediction']]

In [67]:
sales_df_original.shape

(71205, 9)

In [70]:
output_for_David.to_csv('output_sample.csv')